In [326]:
import requests
import pandas as pd
import random

from bs4 import BeautifulSoup

In [327]:
def scrape(url):

    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 OPR/94.0.0.0'})

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

In [328]:
def get_max_page(url):
    soup = scrape(url)

    pages = soup.find('div', class_='pagination-links__container').find_all('a', class_='pagination-item')
    max_page = pages[-1].text.strip()

    return int(max_page)

In [329]:
def get_product(url):
    soup = scrape(url)

    products = soup.find('div', class_='layout__Container-sc-nb1ebc-0 NDIcW')

    attributes = {'Name':[], 'Image Link':[], 'Origin':[], 'Price':[], 'Unit Price':[]}

    for product in products:
        name = product.find('a', class_='link__Link-sc-14ymsi2-0 hSRoYD link__Link-sc-14ymsi2-0 base__Title-sc-1mnb0pd-27 base__FixedHeightTitle-sc-1mnb0pd-43 hSRoYD ifdXVr cCRJZx')
        #print(soup.select('#main > div:nth-child(2) > div > div > div:nth-child(2) > div > div > div:nth-child(3) > div:nth-child(2) > div:nth-child(1) > div > a'))
        origin = product.find('span', class_='rectangular-badge__RectangularBadge-sc-1k9mcpf-0 base__CountryOfOrigin-sc-1mnb0pd-46 jPSXAR gYAJJo')
        price = product.find('strong', class_='base__Price-sc-1mnb0pd-29 sc-ksdxAp fJKWWj irZyf')
        unit_price = product.find('span', class_='text__Text-sc-6l1yjp-0 standard-promotion__PromotionIntentText-sc-1vpsrpe-2 fop__PricePerText-sc-sgv9y1-5 dLNLFE fAoahS eNYENy')

        if price:
            attributes['Price'].append(price.text.strip())
        else:
            attributes['Price'].append('N/A')

        if name:
            attributes['Name'].append(name.text.strip())
        else:
            attributes['Name'].append('N/A')

        if origin:
            attributes['Origin'].append(origin.text.strip())
        else:
            attributes['Origin'].append('N/A')

        if unit_price:
            attributes['Unit Price'].append(unit_price.text.strip())
        else:
            attributes['Unit Price'].append('N/A')
        
        attributes['Image Link'].append('N/A')  

    # if image_link:
    #     attributes['Image Link'].append(image_link['href'])
    # else:
    #     attributes['Image Link'].append('N/A')
    
    return attributes

In [330]:
urls = ['https://plus.casino.fr/products?source=navigation&sublocationId=f1ebadc3-800e-4b99-b69b-bfb99fef00a7']

for url in urls:
    product_dict = get_product(url)
    retail_df = pd.DataFrame(product_dict)

[]


In [325]:
retail_df['Image Link'][0]

'N/A'

In [267]:
retail_df

,Name,Image Link,Origin,Price,Unit Price
0,Casino Bio Cranberries 100G Co Bio,[[[Casino Bio Cranberries 100G Co Bio 1 mois d...,N/A,N/A,N/A
1,Max Havelaar Banane Bio x5,[[[Max Havelaar Banane Bio x5 2 jours de garan...,Pays d’origine Équateur,"1,99 €","(0,40 € / article)"
2,Concombre Bio 1 unité,[[[Concombre Bio 1 unité 2 jours de garantie d...,Pays d’origine Espagne,"1,99 €","(1,99 € / article)"
3,Bouquet Bio Tomates cerise Bio 250g,[[[Bouquet Bio Tomates cerise Bio 250g 3 jours...,Pays d’origine Italie,"2,99 €","(11,96 € / kilo)"
4,Casino Banane Bio Max Havelaar 5 fruits C2,[[[Casino Banane Bio Max Havelaar 5 fruits C2 ...,Pays d’origine République dominicaine,"2,69 €","(3,84 € / kilo)"
...,...,...,...,...,...
79,N/A,N/A,N/A,N/A,N/A
80,N/A,N/A,N/A,N/A,N/A
81,N/A,N/A,N/A,N/A,N/A
82,N/A,N/A,N/A,N/A,N/A


'Nom': name,
'Image': image_link,
'Origine': origin,
'Prix': price,
'Prix à l\'unité': price_per_unit